In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from splink import Splink
from splink.jar_fallback import jw_sim_py

In [2]:
print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("Splink") \
    .master("local[*]") \
    .getOrCreate()


print(spark.sparkContext.appName)

df1 = spark.read.option("header", "true").csv("/home/abhishekks446/data/demo_20.csv")


PySpark Application Started


22/08/02 12:41:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/08/02 12:41:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Splink


In [10]:
settings_with_blocking = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.venue = r.venue and l.year = r.year",
        "l.venue = r.venue and l.title = r.title"

    ],
    "comparison_columns": [

        {
            "col_name": "title",
            "num_levels": 3,
            "term_frequency_adjustments": True
        },
         {
            "col_name": "venue",
          
        }
       
    ]
    ,"em_convergence": 0.09,
        "max_iterations": 98320

}

linker = Splink(settings_with_blocking,df1, spark)
df_e = linker.get_scored_comparisons()

ValidationError: There is an error in your settings dictionary. To quickly write a valid settings dictionary using autocompelte you might want to try our online tool https://moj-analytical-services.github.io/splink_settings_editor/ or you can use the autocomplete features of VS Code - just copy and paste code from the following gist into VS Code, setting language mode to json, or having saved the file as a .json file
https://gist.github.com/RobinL/cfe1152dbd33ae26e05a43d9a0ec85b9

The details of the error are as follows:
0.09 is greater than the maximum of 0.05

Failed validating 'maximum' in schema['properties']['em_convergence']:
    {'$id': '#/properties/em_convergence',
     'default': 0.0001,
     'description': 'The algorithm will stop converging when the maximum '
                    'of the change in model parameters between iterations '
                    'is below this value',
     'examples': [0.0001, 1e-05, 1e-06],
     'maximum': 0.05,
     'minimum': 1e-12,
     'title':

In [6]:
df_new=df_e.toPandas().sort_values("match_probability", ascending=False)

In [7]:
df_new.head(40)

,match_weight,match_probability,unique_id_l,unique_id_r,title_l,title_r,gamma_title,venue_l,venue_r,gamma_venue,match_key
1,2.331856,0.834287,8,9,Report on DART '96: Databases: Active and Real...,Report on DART '96: Databases: Active and Real...,2,SIGMOD Record,SIGMOD Record,1,0
2,1.947533,0.794118,13,14,WaveCluster: A Wavelet Based Clustering Approa...,A Wavelet Based Clustering Approach for Spatia...,1,VLDB J.,VLDB J.,1,0
0,-0.859822,0.355263,5,6,Efficient Geometry-based Similarity Search of ...,Efficient Geometry-3D Spatial Databases,0,SIGMOD Record,SIGMOD Record,1,0
3,-0.859822,0.355263,16,17,Snowball: A Prototype System for Extracting Re...,Snowball: A Prototype,0,SIGMOD Conference,SIGMOD Conference,1,0


In [15]:
settings_without_blocking = {
    "link_type": "dedupe_only",
    "comparison_columns": [

        {
            "col_name": "title",
            "num_levels": 3,
            "term_frequency_adjustments": True
        },
         {
            "col_name": "venue",
          
        }
       
    ]

}

linker = Splink(settings_without_blocking,df1, spark)
df_withoutblocking = linker.get_scored_comparisons()

/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/case_statements.py:20: UserWarning: 

Custom string comparison functions such as jaro_winkler_sim are not available in Spark
Or you did not pass 'spark' (the SparkSession) into 'Model' 
You can import these functions using the scala-udf-similarity-0.0.9.jar provided with Splink.
You will need to add it by correctly configuring your spark config
For example in Spark 2.4.5

from pyspark.sql import SparkSession, types
from pyspark.context import SparkConf, SparkContext
conf.set('spark.driver.extraClassPath', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar') # Not needed in spark 3
conf.set('spark.jars', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar')
spark.udf.registerJavaFunction('jaro_winkler_sim','uk.gov.moj.dash.linkage.JaroWinkle

In [16]:
df_X=df_withoutblocking.toPandas().sort_values("match_probability", ascending=False)

In [17]:
df_X.head(40)

,match_weight,match_probability,unique_id_l,unique_id_r,title_l,title_r,gamma_title,venue_l,venue_r,gamma_venue
121,35.430874,1.000000,13,14,WaveCluster: A Wavelet Based Clustering Approa...,A Wavelet Based Clustering Approach for Spatia...,1,VLDB J.,VLDB J.,1
95,-1.712407,0.233805,11,17,Aurora: A Data Stream Management System,Snowball: A Prototype,0,SIGMOD Conference,SIGMOD Conference,1
65,-1.712407,0.233805,10,2,Dual-Buffering Strategies in Object Bases,Incremental Maintenance for Non-Distributive A...,0,VLDB,VLDB,1
35,-1.712407,0.233805,1,18,Estimation of Query-Result Distribution and it...,Parametric Query Optimization,0,VLDB J.,VLDB J.,1
37,-1.712407,0.233805,2,3,Incremental Maintenance for Non-Distributive A...,Cost-based Selection of Path Expression Proces...,0,VLDB,VLDB,1
38,-1.712407,0.233805,2,4,Incremental Maintenance for Non-Distributive A...,Benchmarking Spatial Join Operations with Spat...,0,VLDB,VLDB,1
160,-1.712407,0.233805,16,17,Snowball: A Prototype System for Extracting Re...,Snowball: A Prototype,0,SIGMOD Conference,SIGMOD Conference,1
44,-1.712407,0.233805,3,4,Cost-based Selection of Path Expression Proces...,Benchmarking Spatial Join Operations with Spat...,0,VLDB,VLDB,1
150,-1.712407,0.233805,15,18,SPRINT: A Scalable Parallel Classifier for Dat...,Parametric Query Optimization,0,VLDB J.,VLDB J.,1
108,-1.712407,0.233805,12,15,Priority Assignment in Real-Time Active Databases,SPRINT: A Scalable Parallel Classifier for Dat...,0,VLDB J.,VLDB J.,1
